## Import Libraries

In [1]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from matplotlib import pyplot as plt
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image
from sklearn.cross_validation import KFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Logistic Regression
from sklearn import datasets
from sklearn import metrics

In [2]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

## label the Images and store into respective folders

In [3]:
directory = "Male"
    
if os.path.exists(directory):
    shutil.rmtree(directory)
    os.makedirs(directory)     
else:
    os.makedirs(directory) 

directory1 = "Female"

if os.path.exists(directory1):
    shutil.rmtree(directory1)
    os.makedirs(directory1)     
else:
    os.makedirs(directory1)     

directory2 = "Label_Images_Gender"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)     
else:
    os.makedirs(directory2)     
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            new_file_extension = prof['Gender']
            new_file_extension = new_file_extension.title()
            #os.rename(fp, filename+"."+new_file_extension)
            copyfile(filename+".jpg", './Label_Images_Gender/'+ uid + '.' + str(id) + "."+new_file_extension +'.jpg')
            copyfile(filename+".jpg", new_file_extension +'/' + uid + ".jpg")

In [4]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [6]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    gender =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        gender_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                edge_images = cv2.Canny(resized,100,200)
                
                images.append(np.hstack((resized.reshape(1,10000), edge_images.reshape(1,10000))))
                labels.append(nbr)

                if gender_current == 'Male':
                    gender.append(1)
                else:
                    gender.append(0)

                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(1)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print gender
    
    return images, labels, gender

In [7]:
images, labels, gender = get_images_and_labels('Label_Images_Gender')

cv2.destroyAllWindows()

1563
2106
1380
1426
2185
3139
2489
3251
1421
2242
986
1349
2084
2952
2446
656
1255
2613
3063
1896
2382
2571
452
901
1112
1780
1852
1823
3232
3000
2454
2195
1531
1835
2353
201
1952
230
2380
633
540
1611
1367
939
2017
1307
3255
960
1554
1009
2505
2988
2277
858
1325
2375
261
548
1360
2124
2986
1070
1848
886
620
634
665
1123
1499
3236
1551
2902
1166
910
53
1288
325
559
2431
2635
260
314
379
1115
710
1094
1778
19
2449
593
2976
2982
1453
1491
479
2963
3010
2244
772
2557
1276
282
2457
105
3092
2729
421
526
146
2754
2562
2494
931
1435
2955
3151
1100
2693
2433
2408
175
583
2995
266
3196
1971
475
1607
1287
837
2211
792
2116
2568
1947
3057
50
1795
1979
699
1400
1319
788
1237
3280
3029
1577
120
178
61
231
319
1756
2141
2526
640
1550
258
1252
391
3195
1732
2175
196
426
1118
1161
1424
799
1294
2700
1244
2637
1480
1148
740
3146
2855
542
2499
361
2650
1866
1444
1553
3017
1089
3112
1711
1670
1398
3258
1894
355
1412
3289
2744
2790
2833
1975
2724
481
2883
3303
1374
2063
267
2075
641
250
2608
243
1602
231

In [40]:
# img = cv2.imread('ankitanarula.jpg',0)
# resized = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
# edge_images = cv2.Canny(resized,100,200)
# #edges = cv2.Canny(img,100,200)
# plt.subplot(121),plt.imshow(resized,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edge_images,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.show()
# edge_images

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
images[0].shape

(1, 20000)

## Making the Dataset 

In [9]:
res_images = []
res_gender = []

#for img in images:
#    res_images.append(img.reshape(1,20000))    # (100*100 image size)

for gen in gender:
    res_gender.append(np.array(gen).reshape(1,))
    
res_images = np.concatenate(images, axis=0)
res_gender = np.concatenate(res_gender,axis=0)

In [10]:
res_images.shape

(2855, 20000)

In [21]:
X,y = res_images,res_gender

(2855, 20000)

## Test the Model for Gender with Edge

In [24]:
def predict(X,y):

    kf = KFold(X.shape[0], n_folds=10, shuffle=True)
    accuracy = 0.0
    precision = [0, 0]
    recall = [0, 0]
    f_measure = [0, 0]

    lr = LogisticRegression()
    inbuilt_accuracy_score = 0.0

    for test, train in kf:
        X_Train, X_Test = X[train], X[test]
        Y_Train, Y_Test = y[train], y[test]

        lr.fit(X_Train, Y_Train)

        pred = lr.predict(X_Test)

        inbuilt_accuracy_score += accuracy_score(Y_Test, (pred))

    #precision = map(lambda x: x / 10, precision)
    #recall = map(lambda x: x / 10, recall)
    #f_measure = map(lambda x: x / 10, f_measure)
    #accuracy /= 10
    #print "Confusion Matrix : \n\n", cm

    inbuilt_accuracy_score /= 10

    #print "*" * 100
    #print "\t\t\t Inbuilt Method"

    print "\nAccuracy from inbuilt Method: ", inbuilt_accuracy_score
    print(metrics.classification_report(Y_Test, pred))
    print(metrics.confusion_matrix(Y_Test, pred))


In [25]:
predict(X,y)


Accuracy from inbuilt Method:  0.638843983866
             precision    recall  f1-score   support

          0       0.57      0.59      0.58      1144
          1       0.66      0.64      0.65      1426

avg / total       0.62      0.62      0.62      2570

[[680 464]
 [511 915]]


## Test the Model for Gender without Edge

In [26]:
X_without_edge,y = res_images[:,:10000],res_gender
X_without_edge.shape

(2855, 10000)

In [27]:
predict(X_without_edge,y)


Accuracy from inbuilt Method:  0.701654067579
             precision    recall  f1-score   support

          0       0.63      0.64      0.64      1146
          1       0.71      0.70      0.70      1424

avg / total       0.67      0.67      0.67      2570

[[735 411]
 [432 992]]


In [26]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('jeffweiner08.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()